# Robot localization using particle filter
The robot has steering and velocity control inputs. It has sensors that measures distance to visible landmarks. Both the sensors and control mechanism have noise in them, and we need to track the robot's position.

In [1]:
import copy
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.animation as animation

from scipy import stats
from IPython.display import HTML
from ipywidgets import interactive
from matplotlib.patches import Circle

# uncomment these two lines if you don't want multiple output in a cell
# just for the convenience of debugging
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

## Constuct particles
Particles can be constructed by randomly sampling in the 2D space, or by Gaussian sampling in place nearby.

In [2]:
def uniform_particles_construction(width, height, N):

    # set the random seed so that we have reproducible experiments
    np.random.seed(500)

    particles = np.random.uniform([0,0], [width, height], size=(N, 2))
    return particles

In [3]:
def gaussian_particles_construction(width, height, N, sigma):
    particles = sigma * np.random.randn(N, 2) + (width/2, height/2)
    return particles

## Reject sampled particles
Here we use the random sampling method as illustration. Since previously we get particles randomly among the total 2D spaces, it is possible that there are some particles located inside landmarks, thus we neet to delete these invalid particles.

In [4]:
def rejection_particles(particles, centers, radius):
    """Given randomly sampled particles and centers of landmarks, perform rejection here
    
    Args:
        particles: the particles we get through random generation in 2D space
        centers: centers of landmarks
        radius: the radius of cicular landmarks 
    """
    particles_after_rejection = []
    for count_p, coord_p in enumerate(particles):
        dis = np.linalg.norm(coord_p-centers, axis=1, keepdims=True)
        if np.all(dis >= radius):
            particles_after_rejection.append(particles[count_p])
    return np.asarray(particles_after_rejection)

Now let's predefine some parameters and then run two examples to see the influence of number of evidence. What do you see in these two examples?

In [5]:
[width_max, height_max]= [800, 600]

# Landmark center coordinates

# Version 1: 5 landmarks
landmark_centers = np.array([ [336,175], [718,159], [510,43], [167, 333], [472, 437] ])
landmark_radii = np.array([[12],[6],[7],[18],[9]])

# Version 2: ten landmarks
# centers = np.array([ [144,73], [510,43], [336,175], [718,159], [178,484], [665,464], [267, 333], [541, 300], [472, 437], [100, 533] ])
# radii = np.array([[12],[32],[7],[8],[13],[6],[7],[8],[9],[10]])

if len(landmark_centers) != len(landmark_radii):
    raise ValueError("centers and radii must have the same size!")

num_particles = np.arange(1000, 10000, 1000)

for i in range(len(num_particles)):
    particles = uniform_particles_construction(width_max, height_max, num_particles[i])
    rejection =  rejection_particles(particles, landmark_centers, landmark_radii)
    print('Estimated acceptability for {} particles is {:.5f}'.format(num_particles[i], len(rejection[:,0])/len(particles[:,0])))

print('The true acceptability is {:.5f}'.format(1 - (np.pi * landmark_radii * landmark_radii).sum() / (width_max * height_max) ))

Estimated acceptability for 1000 particles is 0.99700
Estimated acceptability for 2000 particles is 0.99750
Estimated acceptability for 3000 particles is 0.99767
Estimated acceptability for 4000 particles is 0.99750
Estimated acceptability for 5000 particles is 0.99700
Estimated acceptability for 6000 particles is 0.99717
Estimated acceptability for 7000 particles is 0.99657
Estimated acceptability for 8000 particles is 0.99625
Estimated acceptability for 9000 particles is 0.99644
The true acceptability is 0.99585


## Motion model
Now we can move the remaining particles based on how you predict the real system is behaving with some noise in the motion model. Assuming that out sensors returns the data in a time interval of 0.5s.

In [6]:
def predict_particle_pos(particles, v, std, dt=0.5):
    """Predict the motion of next state for each particles given current angles and velocities.
    
    Args:
        particles: the particles we get after rejecting the ones that are not available
        v： 2d array. Each sample with feature [angle, velocity]
        std: standard deviation of velociy
        dt: time interval, assume it to be 0.5 second here
    """
    N = len(particles)
    
    # std can be set as a hyperparameter to decide how noisy is the data, thus can change difficulty
    delta_dist = (v[1] * dt) + (np.random.randn(N) * std) # add some noise to the distance
    particles[:, 0] += np.cos(v[0]) * delta_dist
    particles[:, 1] += np.sin(v[0]) * delta_dist

## Update the weights of each particle
Update the weighting of the particles based on the measurement. Each particle has a position and a weight which estimates how well it matches the measurement. Normalizing the weights so they sum to one. This normalization step turns them into a probability distribution. Those particles that are closer to the robot will generally have a higher weight than ones far from the robot. Particles that closely match the measurements are weighted higher than particles which don't match the measurements very well. So in this case, we can measure the probability using the distance to landmarks.

In [7]:
def weights_update(particles, weights, dist_r_l, centers, radii, scale_fac, std):
    """Given the noised distances from robot to the landmarks, update the weights of particles
    
    Args:
        particles: coordinate of particles
        weights: weight of particles
        dist_r_l: the current distance between robot and landmarks
        scale_fac, std: hyperparameters to avoid the underflow of possibilities
    """
    
    weights.fill(1.)
    
    for count, center in enumerate(centers):
        # distance between the particles and each landmark
        dist_p_l = np.linalg.norm(particles-center, axis=1, keepdims=True) - radii[count]
        
        # have tried use exponential function to avoid underflow, but still of no use
        # since the distance is in the range of 1e2, so the possibility is 0 everywhere
        # so here use scale_fac and std to avoid the underflow of possibilities
        # set the distance as mean and std as standard deviation of norm distribution, then get the pdf as our new weights
        weights *= stats.norm.pdf(dist_p_l/scale_fac, dist_r_l[count]/scale_fac, std)

    weights += 1.e-100   # avoid round-off to zero
    weights /= sum(weights)

# Resample Procedures
Discard highly impossible particles and replace them with copies of the more possible particles. Accordingly, particles with greater weights survive with higher likelihood than particles with valus of small importance. In principle, there are many ways to achieve this, here you can refer to the procedure given in the paper [Resampling in Particle Filtering - Comparison](http://sait.cie.put.poznan.pl/38/SAIT_38_02.pdf) to complete the systematic resampling method, in which the algorithm is described as below:
<img src="img/systematic_resample.png" alt="Encoder" style="width: 400px;"/>
This resampling has a complexity of *O(N)*, and is one of the more readily recommended, because of its simplicity and operation speed. This approach assumes that the range $[0, 1)$ is subdivided in to N equal parts, and the draw occurs in each stratum
$u^i \sim [\frac{i-1}{N}, \frac{i}{N})$, particles are selected for replication, such that $u^i \in [\sum_p^{j-1}q_p, \sum_p^{j}q_p)$

In [8]:
def resample_procedure(weights):
    """Perform resampling procedure described above
    
    Args:
        weights: the weights to be updated
    
    ReturnL:
        idx: the indices of those remained particled
    """
    
    
    num_weights = len(weights)
    
    # make N subdivisions, choose positions with a consistent random offset
    U = (np.arange(num_weights) + np.random.uniform()) / num_weights
 
    idx = np.zeros(num_weights, 'i') # set the data type as int
    sum_Q = np.cumsum(weights)
    
    i, j = 0, 0
    while i<num_weights and j<num_weights:
        if U[i] < sum_Q[j]:
            idx[i] = j
            i += 1
        else:
            j += 1
    return idx

The function above takes an array of weights and returns indexes of particles that have been chosen. We just need to write a function that performs the resampling from these indexes:

In [9]:
def resample_from_index(particles, weights, idx):
    particles[:] = particles[idx]
    weights[:] = weights[idx]
    weights /= np.sum(weights)

Now let's put the prediction positions of these particles together

# Load the data

In [10]:
# input the origin data as reference
# you should not touch this data in you implementation
input_sequence = np.load('./archive/Trajectory_1.npy')
# Now let's input the velocity and distance data
# So should be intepreted as corresponding transition and observability matrix in HMM type
angle_velocity = np.load('./data/velocity_1.npy')
dist_r_l = np.load('./data/distance_1.npy')
# Now need to figure out what can be done next if you have velocity and diatance data at hand

## Visulization
Now we visualize the moving of the robot to show how your particle filters works! What we need to achieve here is adding the moving of input sequence as well as particles onto that map.

In [11]:
# set hyperparametes
original_particle_number = 500
gaussian_particle_std = 100

# add noise to the distance prediction
dis_std = 2

# decide the convergence of particles
weights_scale = 50 
weights_std = 1

In [15]:
# Then we get the particles
random_particles = uniform_particles_construction(width_max, height_max, original_particle_number)
reject_random_particles = rejection_particles(random_particles, landmark_centers, landmark_radii)
origin_random_weights = np.ones((len(reject_random_particles),1))

# Now we need to record the coordinates of moving particles
prediction_random_paticles = [reject_random_particles]

random_pos = copy.copy(reject_random_particles)
random_weights = copy.copy(origin_random_weights)

for i in range(len(dist_r_l)):

    # The predicted position of particles
    predict_particle_pos(random_pos, angle_velocity[i], dis_std, 0.5)
    weights_update(random_pos, random_weights, dist_r_l[i], landmark_centers, landmark_radii, weights_scale, weights_std)
    random_index = resample_procedure(random_weights)
    resample_from_index(random_pos, random_weights, random_index)
    prediction_random_paticles.append(copy.copy(random_pos))
    
particle_random_trajectory = np.asarray(prediction_random_paticles)

In [19]:
def location(step):
    img = plt.imread('img/Canvas.png')
    fig,ax = plt.subplots(1)

    # Now, loop through coord arrays, and create a circle at center
    for count, value in enumerate(landmark_centers):
        circ = Circle(value,landmark_radii[count])
        ax.add_patch(circ)
    
    ax.scatter(input_sequence[step,0], input_sequence[step,1], s=15, c='r', marker='x')
    ax.scatter(particle_random_trajectory[step,:,0], particle_random_trajectory[step,:,1],s=3, c='g')
    
    plt.xlim(0, width_max) 
    plt.ylim(0, height_max)

    # Create a figure. Equal aspect so circles look circular
    ax.set_aspect('equal')
    ax.imshow(img)

In [20]:
iplot = interactive(location, step=(0, len(particle_random_trajectory)-1))
iplot

interactive(children=(IntSlider(value=14, description='step', max=28), Output()), _dom_classes=('widget-intera…

We see that the robot starts at zero point, but the particles sampled randomly in the 2D spaces rarely go close to the zero, hence cause the bias which exists all over the trajectory. So instead, let's try with the Gaussian sample menthod.

In [23]:
gaussian_particles = gaussian_particles_construction(width_max, height_max, original_particle_number, gaussian_particle_std)
reject_gaussian_particles = rejection_particles(gaussian_particles, landmark_centers, landmark_radii)
origin_gaussian_weights = np.ones((len(reject_gaussian_particles),1))

# Now we need to record the coordinates of moving particles
prediction_gaussian_paticles = [reject_gaussian_particles]

gaussian_pos = copy.copy(reject_gaussian_particles)
gaussian_weights = copy.copy(origin_gaussian_weights)

for i in range(len(input_sequence)-1):

    predict_particle_pos(gaussian_pos, angle_velocity[i], dis_std, 0.5)
    weights_update(gaussian_pos, gaussian_weights, dist_r_l[i], landmark_centers, landmark_radii, weights_scale, weights_std)
    gaussian_index = resample_procedure(gaussian_weights)
    resample_from_index(gaussian_pos, gaussian_weights, gaussian_index)
    prediction_gaussian_paticles.append(copy.copy(gaussian_pos))
    
particle_gaussian_trajectory = np.asarray(prediction_gaussian_paticles)

In [31]:
def location(step):
    img = plt.imread('img/Canvas.png')
    fig,ax = plt.subplots(1)

    # Now, loop through coord arrays, and create a circle at center
    for count, value in enumerate(landmark_centers):
        circ = Circle(value,landmark_radii[count])
        ax.add_patch(circ)
    
    ax.scatter(input_sequence[step,0], input_sequence[step,1], s=15, c='r', marker='x')
    ax.scatter(particle_gaussian_trajectory[step,:,0], particle_gaussian_trajectory[step,:,1],s=1, c='g')
    
    plt.xlim(0, width_max) 
    plt.ylim(0, height_max)

    # Create a figure. Equal aspect so circles look circular
    ax.set_aspect('equal')
    ax.imshow(img)

In [32]:
iplot = interactive(location, step=(0, len(particle_gaussian_trajectory)-1))
iplot

interactive(children=(IntSlider(value=14, description='step', max=28), Output()), _dom_classes=('widget-intera…

Then let's predict the position of robot's final position and make comparison with the true result. The final position should be somewhere around ...(tbd)

In [33]:
predict_final_random = np.average(particle_random_trajectory[-1], weights=random_weights.flatten(), axis=0)
print("The estimate position using random sampled particles is {}".format(predict_final_random))
predict_final_gaussian = np.average(particle_gaussian_trajectory[-1], weights=gaussian_weights.flatten(), axis=0)
print("The estimate position using Gaussian sampled particles is {}".format(predict_final_gaussian))
print("The true final position of robot is {}".format(input_sequence[-1]))

The estimate position using random sampled particles is [416.49524062 227.03834284]
The estimate position using Gaussian sampled particles is [405.28422945 235.12199051]
The true final position of robot is [420.31088083 217.12707182]
